# Fire monitoring tutorial

This tutorial provides an overview of our [open source pipeline](https://github.com/carbonplan/forest-offsets-fires) for generating a cloud-optimized GeoParquet dataset for active and recent fires, along with vector tiles for performant visualization.

## Import packages

We've packaged most of the utilities needed for the data pipeline in the `carbonplan_forest_offsets_fires` Python library.

In [ ]:
from carbonplan_forest_offsets_fires.firms import (
    read_firms_nrt,
    filter_df,
    mask_df,
    make_tile_tempdir,
    write_firms_json,
    build_tippecanoe_cmd,
    build_pbf_cmd,
    upload_tiles,
)
import pandas as pd
import subprocess
from tempfile import TemporaryDirectory

## Define parameters

We define these parameters for creating a cloud-optimized dataset for the continental United States and Alaska.

In [ ]:
day_range = 3
params = {'min_lat': 24, 'max_lat': 72, 'min_lon': -180, 'max_lon': -66, 'day_range': day_range}


## Load and transform data from the FIRMS API

In this section, we use the utilities in CarbonPlan's fire monitoring Python package to query the FIRMS API, subset the data, filter out low confidence data, and transform into a GeoDataFrame.

For this to work, you need to have a valid API token stored in the environment variable ``FIRMS_MAP_KEY``.

In [ ]:
df_snpp = read_firms_nrt(
    **params,
    source="VIIRS_SNPP_NRT",
).pipe(filter_df)
df_noaa20 = read_firms_nrt(
    **params,
    source="VIIRS_NOAA20_NRT",
).pipe(filter_df)
df_modis = read_firms_nrt(
    **params,
    source="MODIS_NRT",
).pipe(filter_df)
df = pd.concat([df_snpp, df_noaa20, df_modis])
gdf = mask_df(df)

## Write output to GeoParquet
In this section, we'll write the output to GeoParquet. We'll write to a temporary directory for this tutorial. For the production pipeline, we write the data to cloud object storage.

In [ ]:
td = TemporaryDirectory()
gdf.to_parquet(f'{td.name}/current-firms-pixels.parquet')


## Create tiles
Now that we have the data in a cloud-optimized format, we'll also create vector tiles for performant viewing in the browser. We'll use the [tippecanoe](https://github.com/mapbox/tippecanoe) library to build the tiles. The output from these are written to temporary directories. In the production workflow, we upload the Protocol Buffer files derived from vector tiles to AWS S3 for rendering in the [monitoring forest offsets and fires web tool](https://carbonplan.org/research/forest-offsets-fires).

In [ ]:
STEM = 'current-firms-pixels'
# Create temporary GeoJSON that can be read by tippecanoe
tempdir = make_tile_tempdir()
json_fp = write_firms_json(data=gdf, tempdir=tempdir)
# Use tippecanoe to generate vector tiles
tippecanoe_cmd = build_tippecanoe_cmd(input_fn=json_fp, tempdir=tempdir, stem=STEM)
subprocess.run(tippecanoe_cmd)
# Use mb-util to generate pbf files from vector tiles
pbf_cmd = build_pbf_cmd(tempdir=tempdir, stem=STEM)
subprocess.run(pbf_cmd)
